# Hackaton La Rondelle

In [2]:
import pandas as pd
import xarray as xr

ds = xr.open_dataset("data/OSSE_U_V_SLA_SST_train.nc")
ds

<xarray.Dataset> Size: 1GB
Dimensions:       (latitude: 357, longitude: 717, time_counter: 284)
Coordinates:
    deptht        float32 4B ...
  * latitude      (latitude) float32 1kB -1.084e-19 2.967 0.0 ... 3.099 0.0
  * longitude     (longitude) float32 3kB 3.689e+19 -3.352 0.0 ... -3.221 0.0
  * time_counter  (time_counter) datetime64[ns] 2kB 2015-01-02T12:00:00 ... 2...
Data variables:
    vomecrtyT     (time_counter, latitude, longitude) float32 291MB ...
    vozocrtxT     (time_counter, latitude, longitude) float32 291MB ...
    sossheig      (time_counter, latitude, longitude) float32 291MB ...
    votemper      (time_counter, latitude, longitude) float32 291MB ...
Attributes:
    history:  Mon Feb 20 14:04:07 2023: ncks -A SST_2015_merged.nc SSH_U_V_20...
    NCO:      4.4.7

In [5]:
df = ds.to_dataframe()
df

deptht  vomecrtyT  \
latitude      longitude    time_counter                               
-1.084202e-19 3.689349e+19 2015-01-02 12:00:00  0.494025        NaN   
                           2015-01-03 12:00:00  0.494025        NaN   
                           2015-01-04 12:00:00  0.494025        NaN   
                           2015-01-05 12:00:00  0.494025        NaN   
                           2015-01-06 12:00:00  0.494025        NaN   
...                                                  ...        ...   
 0.000000e+00 0.000000e+00 2015-12-21 12:00:00  0.494025   0.189014   
                           2015-12-22 12:00:00  0.494025   0.273300   
                           2015-12-23 12:00:00  0.494025   0.267159   
                           2015-12-24 12:00:00  0.494025   0.243072   
                           2015-12-25 12:00:00  0.494025   0.215646   

                                                vozocrtxT  sossheig  votemper  
latitude      longitude    time_counter                                        
-1.084202e-19 3.689349e+19 2015-01-02 12:00:00        NaN       NaN       NaN  
                           2015-01-03 12:00:00        NaN       NaN       NaN  
                           2015-01-04 12:00:00        NaN       NaN       NaN  
                           2015-01-05 12:00:00        NaN       NaN       NaN  
                           2015-01-06 12:00:00        NaN       NaN       NaN  
...                                                   ...       ...       ...  
 0.000000e+00 0.000000e+00 2015-12-21 12:00:00  -0.005987 -0.860745  8.453655  
                           2015-12-22 12:00:00   0.031291 -0.863894  8.370595  
                           2015-12-23 12:00:00   0.070180 -0.869595  8.129839  
                           2015-12-24 12:00:00   0.116377 -0.877435  8.155015  
                           2015-12-25 12:00:00   0.038586 -0.822859  8.354387  

[72695196 rows x 5 columns]

In [ ]:
ds.close()